In [1]:
import matplotlib.pyplot as plt
import math
import numpy as np
from timeit import default_timer
import numba
from numba import jit, cuda, vectorize, guvectorize
from numba import void, uint8 , uint32, uint64, int32, int64, float32, float64, f8

# Esercizi

1. Scrivi un kernel che, dati due vettori x,y calcoli il vettore z definito come
    $$ z[i] = x[i]^2\cos(y[i])$$
    Fai in confronto con Numpy, con la stessa funzione scritta in puro Python e con vectorize. Cosa puoi dire delle tempistiche?
    Qual è lo speed-up oppetuto rispetto a queste due implementazioni?

3. Completare esercizio sulla regione di stabilità della settimana 2.

### 1.

In [2]:
# Versione CPU
@numba.vectorize(['float32(float32, float32)',
                  'float64(float64, float64)'], target='cpu')
def sqcos_cpu(x, y):
    return x**2 * math.cos(y)

# Versione CUDA
@numba.vectorize(['float32(float32, float32)',
                     'float64(float64, float64)'], target='cuda')
def sqcos_cuda(x, y):
    return x**2 * math.cos(y)

# Versione Numpy
def sqcos_np(x,y):
  return x**2 * np.cos(y)

# Versione CUDA kernel
@cuda.jit
def sqcos_ker(x, y, z):
    tx = cuda.threadIdx.x # indice del thread nel blocco
    bx = cuda.blockIdx.x  # indice del blocco nella griglia
    bw = cuda.blockDim.x  # numero di threads per blocco
    i = tx + bx * bw # indice del thread nella griglia

    if i < z.size:
        z[i] = x[i]**2 * math.cos(y[i])
    #endif
    return

In [3]:
# Generazione dei dati
n = 1000000
x = np.linspace(10, 10, n)
y = np.linspace(0, 2*np.pi, n)
z = np.empty_like(x)

threads_per_block = 128
blocks_per_grid= int(math.ceil(float(n) / threads_per_block))

In [4]:
# Speed-up
# np
t_i = default_timer()
sqcos_np(x, y)
t_f = default_timer()
np_time = t_f -t_i
# cpu
t_i = default_timer()
sqcos_cpu(x, y)
t_f = default_timer()
cpu_time = t_f -t_i
# gpu
t_i = default_timer()
sqcos_cuda(x, y)
t_f = default_timer()
cuda_time = t_f -t_i
# ker
t_i = default_timer()
sqcos_ker[blocks_per_grid, threads_per_block](x, y, z)
t_f = default_timer()
ker_time = t_f -t_i

print('speed-up di CPU vectorize vs Numpy = ', np_time/cpu_time )
print('speed-up di GPU vectorize vs Numpy = ', np_time/cuda_time )
print('speed-up di Kernel vs Numpy = ', np_time/ker_time )

speed-up di CPU vectorize vs Numpy =  1.0688382646211005
speed-up di GPU vectorize vs Numpy =  0.09660714154343676
speed-up di Kernel vs Numpy =  0.33954461038762235


/usr/local/lib/python3.10/dist-packages/numba/cuda/cudadrv/devicearray.py:885: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))
